In [1]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

import pickle
import os
import torch

import pandas as pd

import plotly.io as pio
pio.renderers.default = 'notebook+vscode'
from sklearn.cluster import KMeans

import plotly.graph_objs as go

from ipywidgets import Output, VBox, Button

#import sys
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/registration/')
#sys.path.insert(0, '/Users/danielverschueren/Documents/Sonalis/code/registration/')

from point_cloud_torch import affinePC_torch, apply_affine_torch, apply_affine_inverse_torch
from point_cloud import plot_PCs, affinePC

# the fit function in affinePC calls on minimize_parallel from optimparallel
# make sure that package is installed
def create_cylinder(z, r):
    """
    construct a point cloud of a cylinder with radius r and length len(z)
    """
    phi = np.linspace(0, 2*np.pi, 100)
    xyz = np.zeros((phi.shape[0]*z.shape[0], 3))
    
    for i in range(phi.shape[0]):
        j = i*z.shape[0]
        xyz[j:j+z.shape[0],0] = np.sin(phi[i])*r 
        xyz[j:j+z.shape[0],1] = np.cos(phi[i])*r
        xyz[j:j+z.shape[0],2] = z

    return xyz

def pgy2npy(pgy_file):
    """
    pgy file to npy
    """
    with open(pgy_file) as f:
        lines = f.readlines()
    lines = lines[1:] # remove header
    np_data = np.zeros((len(lines), 3), np.float64)
    for i, line in enumerate(lines):
        l = []
        for t in line.split():
            try:
                l.append(float(t))
            except ValueError:
                pass
        np_data[i] = np.asarray(l[1:])
    return np_data

def naive_dist(a, b, p):
    """
    ===========================================================================+
    Find matching points from b to a using a minium distance assignment.

    INPUT
    a                           (np.array)(float) point cloud of reference
                                (NUM_PTS_A, 3)
    b                           (np.array)(float) point cloud of reference
                                (NUM_PTS_B, 3)
    p                           (int) order of the p-norm used to calculate 
                                distances between points in a and b.
                       
    OUTPUT
    assignments                 (np.array)(int) mapping of points B to A
                                (NUM_PTS_A,)
    ===========================================================================+
    """

    length_a = a.shape[0]
    length_b = b.shape[0]

    # calculate the benefit matrix between sets ai and bi
    B = np.repeat(b[:,:,np.newaxis],   length_a, axis=2)
    A = np.repeat(a.T[np.newaxis,:,:], length_b, axis=0)
    cost_matrix = (np.linalg.norm(A-B, ord=p, axis=1)**p)

    # find min dist
    d = np.min(cost_matrix, axis=1)
    
    return d

plot_using = 'plotly_light'

## load objects

In [2]:
cap = np.load('coords/cap.npy')
cyl = np.load('coords/cyl.npy')

In [5]:
# refine fitting: remove pointcloud halos from points
labels_fit = np.array([ 442,  546,  685,  993, 1093, 1094, 1124, 1197, 1222, 1248, 1300,
       1302, 1305, 1326, 1351, 1371, 1405, 1431, 1447, 1473, 1481])
flipped = [685] #546
fucked = [1371]
path = ""

n_sub = 10
count = 0 
errors = []
n_real = 1703

## Check the initial fits

In [14]:
# isolate problematic transducers
losses = list()

for i in range(1696):
    path_file = path + f"fittedTransducers/{i}/transCyl_{i}.pk"
    with open(path_file, 'rb') as f:
        x = pickle.load(f)
    loc = x['location']
    A_PC = x['Objects']
    loss = A_PC.loss_l2(A_PC.params, A_PC.pc_ct, A_PC.pc_us, A_PC.center_ct, method='naive')/len(A_PC.pc_us)
    losses.append(loss)

lossArgs = np.argwhere(np.array(losses) > 0.0003).T[0]

In [25]:
print(lossArgs)
print(len(lossArgs))

[  10   15   16   19   24   30   31   33   38   44   49   50   51   53
   70   75   83   86   88   92   95  105  108  116  118  120  128  129
  138  158  160  174  178  180  182  185  187  197  199  200  201  209
  210  211  217  220  223  225  228  232  244  250  252  264  269  270
  273  278  280  286  288  303  305  307  313  335  338  341  349  352
  353  354  363  365  367  369  371  372  379  389  398  403  419  424
  438  441  442  449  453  462  467  472  473  477  478  482  486  495
  511  518  529  534  538  539  554  568  572  575  576  589  604  605
  609  613  629  631  635  638  640  642  643  656  657  668  673  687
  688  690  693  696  706  710  712  720  727  728  734  741  754  756
  758  761  774  777  786  793  796  803  821  828  832  860  867  869
  870  874  884  890  893  898  927  945  947  959  963  964  965  966
  970  987  988  990  997 1001 1004 1010 1012 1021 1025 1026 1028 1029
 1032 1033 1034 1041 1043 1060 1064 1069 1075 1077 1099 1107 1117 1118
 1123 

In [ ]:
# test a fit
for i in lossArgs[:2]:
    path_file = path + f"fittedTransducers/{i}/transCyl_{i}.pk"
    with open(path_file, 'rb') as f:
        x = pickle.load(f)
    loc = x['location']
    A_PC = x['Objects']
    loss = A_PC.loss_l2(A_PC.params, A_PC.pc_ct, A_PC.pc_us, A_PC.center_ct, method='naive')
    print(f"{i}:  {loss/len(A_PC.pc_us)}")

    # plot distances
    temp = A_PC.pc_us
    distance = naive_dist(A_PC.pc_ct_r, temp, 2)
    plt.plot(np.sort(distance))
    plt.grid()
    plt.show()

    # cure
    x = temp[:,0]
    y = temp[:,1]
    z = temp[:,2]
    temp_p = temp[distance < 0.2]
    fig = plot_PCs([A_PC.pc_us, temp_p, A_PC.pc_ct_r])
    fig.show()

    clean_temp = temp[(z > -3) & (x < 1.3)]
    fig = plot_PCs([temp, clean_temp])
    fig.show()

## remove points from cloud

In [33]:
# Create the scatter plot
label = lossArgs[14]
print(label)
path_file = path + f"fittedTransducers/{label}/transCyl_{label}.pk"
with open(path_file, 'rb') as f:
    x = pickle.load(f)
loc = x['location']
A_PC = x['Objects']
loss = A_PC.loss_l2(A_PC.params, A_PC.pc_ct, A_PC.pc_us, A_PC.center_ct, method='naive')
print(f"{label}:  {loss/len(A_PC.pc_us)}")

fig = go.FigureWidget()
pie = fig.add_scatter3d(x=A_PC.pc_us[:,0], y=A_PC.pc_us[:,1], z=A_PC.pc_us[:,2], mode='markers')

out = Output()
@out.capture(clear_output=True)
def remove_point(trace, points, selector):
    print(f"Clicked point indices: {points}")
    x_new = [point for i, point in enumerate(fig.data[0].x) if i not in points.point_inds]
    y_new = [point for i, point in enumerate(fig.data[0].y) if i not in points.point_inds]
    z_new = [point for i, point in enumerate(fig.data[0].z) if i not in points.point_inds]

    with fig.batch_update():
        fig.data[0].x = x_new
        fig.data[0].y = y_new
        fig.data[0].z = z_new

def save_cloud(_):
    remaining_points = list(zip(fig.data[0].x, 
                                fig.data[0].y, 
                                fig.data[0].z))
    with open(path + f"fittedTransducers/{label}/pc-{label}-refined.csv", "w") as f:
        for point in remaining_points:
            f.write(f"{point[0]}, {point[1]}, {point[2]}\n")
    print(f"Remaining points saved to 'pc-{label}-refined.csv'")

# Create the 'Save Cloud' button
save_button = Button(description="Save Cloud")
save_button.on_click(save_cloud)
pie.data[0].on_click(remove_point)


VBox([fig, save_button, out])

70
70:  0.0003942705719103632


    'data': [{'mode': 'markers',
              'type': 'scatter3d',
            …

Remaining points saved to 'pc-70-refined.csv'


In [ ]:
# fix 1053: split in 2
for i in [685]:

    path_file = f"fittedTransducers-1/{i}/transCyl_{i}.pk"
    with open(path_file, 'rb') as f:
        x = pickle.load(f)
    loc = x['location']
    A_PC = x['Objects']
    loss = A_PC.loss_l2(A_PC.params, A_PC.pc_ct, A_PC.pc_us, A_PC.center_ct, method='naive')
    print(f"{i}:  {loss/len(A_PC.pc_us)}")

    # plot distances
    temp = A_PC.pc_us
    distance = naive_dist(A_PC.pc_ct_r, temp, 2)
    plt.plot(np.sort(distance))
    plt.grid()
    plt.xlabel('point #')
    plt.ylabel('L2 distance')
    path_file = f"fittedTransducers-1/{i}/error_curve.png"
    plt.savefig(path_file)
    plt.close()

    # cure
    z = A_PC.pc_us[:,2]
    print(A_PC.params)
    
    B_PC = affinePC(A_PC.pc_us[z < -2], A_PC.pc_ct)
    B_PC.center_ct = A_PC.center_ct
    dx = 20
    dphi = 0.5
    start=A_PC.params
    start[1] = np.pi/2 + np.pi
    #start[3] = 0
    start[5] = 0
    bounds_t = [(start[3]-dx, start[3]+dx), 
                (start[4]-dx, start[4]+dx),
                (start[5]-dx, start[5]+dx)]
    bounds_r = [(start[0]-dphi, start[0]+dphi), 
                (start[1]-dphi, start[1]+dphi),
                (start[2]-dphi, start[2]+dphi)]
    bounds = bounds_r + bounds_t
    B_PC.fit(start=start, bounds=bounds, method='naive')
    print(f"Affine parametes [rot_x, rot_y, rot_z, t_x, t_y, t_z]: \n{B_PC.params}")
    regrefined_pc = B_PC.apply_aff()
    loss_new = B_PC.loss_l2(B_PC.params, B_PC.pc_ct, B_PC.pc_us, B_PC.center_ct, method='naive')
    print(f"{i} {loss/len(A_PC.pc_us)} vs {loss_new/len(B_PC.pc_us)}")

    # save
    bucket = {"location" : loc,
              "Objects"  : B_PC}
    with open(f"fittedTransducers-1/{i}/transCyl_{i}_refitCutoff.pk", 'wb') as handle:
        pickle.dump(bucket, handle)
    
    fig = plot_PCs([regrefined_pc, A_PC.pc_us, B_PC.pc_us])
    fig.write_html(f"fittedTransducers-1/{1700}/transCyl_{1700}_refitCutoff.html")